In [1]:
#default_exp segmentation.models

In [2]:
%autosave 60 
import fastai; print(fastai.__version__)

Autosaving every 60 seconds
1.0.58.dev0


In [3]:
#export
from fastai.vision import *

### data

In [4]:
from local.segmentation.dataset import SemanticSegmentationData
from local.segmentation.metrics import *
# from local.segmentation.losses_binary import *
from local.segmentation.losses_multilabel import *
# test data creation
PATH = Path("/home/turgutluk/.fastai/data/camvid")
IMAGES = "images"
MASKS = "labels"
CODES = "codes.txt"
TRAIN, VALID, TEST = "train.txt", "valid.txt", "test.txt"
ssdata = SemanticSegmentationData(PATH, IMAGES, MASKS, CODES, TRAIN,
                                  VALID, TEST, sample_size=None, bs=4, size=112)
data = ssdata.get_data()

### configs

In [5]:
#export
default_configs = {}
model_funcs = {}
splits = {}

In [6]:
#export
def get_model(name, data, config):
    "Get model given name, data and config. Undefined config is defaulted."
    conf, copy_conf = default_configs[name].copy(), default_configs[name].copy()
    conf.update(config)    
    f = model_funcs[name]
    model = f(data, conf)
    split = splits.get(name)
    return model, split, copy_conf

In [17]:
#export - dynamic unet config
from fastai.vision.models.unet import DynamicUnet
from fastai.vision.learner import cnn_config

body_config = {"pretrained":True, "cut":None} 
unet_config = {"blur":False, "blur_final":True, "self_attention":False,
         "y_range":None, "norm_type":NormType, "last_cross":True, "bottle":False}
dunet_config = {**body_config, **unet_config}

### `ResDUnet [18,34,50,101,152]`

In [8]:
#export
from fastai.vision.models import resnet18, resnet34, resnet50, resnet101, resnet152

In [10]:
dunet_config

{'pretrained': True,
 'cut': None,
 'blur': False,
 'blur_final': True,
 'self_attention': False,
 'y_range': None,
 'norm_type': <enum 'NormType'>,
 'last_cross': True,
 'bottle': False}

In [11]:
#export
def _resdunet(arch, data, config):
    "Returns a resdunet model for a arch from data and final config"
    pretrained, cut = config.pop("pretrained"), config.pop("cut")
    try:    size = data.train_ds[0][0].size
    except: size = next(iter(data.train_dl))[0].shape[-2:]
    body = create_body(arch, pretrained, cut)
    model = DynamicUnet(body, n_classes=data.c, img_size=size, **config)
    return model

In [12]:
#export
def resdunet18(data, config): return _resdunet(resnet18, data, config)
model_funcs['resdunet18'] = resdunet18
default_configs['resdunet18'] = dunet_config
splits['resdunet18'] = cnn_config(resnet18)['split']

def resdunet34(data, config): return _resdunet(resnet34, data, config)
model_funcs['resdunet34'] = resdunet34
default_configs['resdunet34'] = dunet_config
splits['resdunet34'] = cnn_config(resnet34)['split']

def resdunet50(data, config): return _resdunet(resnet50, data, config)
model_funcs['resdunet50'] = resdunet50
default_configs['resdunet50'] = dunet_config
splits['resdunet50'] = cnn_config(resnet50)['split']

def resdunet101(data, config): return _resdunet(resnet101, data, config)
model_funcs['resdunet101'] = resdunet101
default_configs['resdunet101'] = dunet_config
splits['resdunet101'] = cnn_config(resnet101)['split']

def resdunet152(data, config): return _resdunet(resnet152, data, config)
model_funcs['resdunet152'] = resdunet152
default_configs['resdunet152'] = dunet_config
splits['resdunet152'] = cnn_config(resnet152)['split']

In [13]:
model, split,_ = get_model(name="resdunet50", data=data, config={"self_attention":True})

### `DenseDUnet [121,161,169, 201]`

In [16]:
# from fastai.callbacks.hooks import model_sizes
# from fastai.vision.models import densenet121, densenet161, densenet169, densenet201

In [17]:
# def _densedunet_split(m:nn.Module): return (m[0][3], m[1])

In [64]:
# #export
# def _densedunet(arch, data, config):
#     "Returns a resdunet model for a arch from data and final config"
#     pretrained, cut = config.pop("pretrained"), config.pop("cut")
#     body = create_body(arch, pretrained, cut)[0]
#     densenet_children = list(body.children())
#     new_body = nn.Sequential(nn.Sequential(*densenet_children[:4]),
#                                nn.Sequential(*densenet_children[4:6]),
#                                nn.Sequential(*densenet_children[6:8]),
#                                nn.Sequential(*densenet_children[8:10]),
#                                nn.Sequential(*densenet_children[10:]))
#     try:    size = data.train_ds[0][0].size
#     except: size = next(iter(data.train_dl))[0].shape[-2:]
#     model = DynamicUnet(new_body, n_classes=data.c, img_size=size, **config)
#     return model

In [65]:
# #export
# def densedunet121(data, config): return _densedunet(densenet121, data, config)
# model_funcs['densedunet121'] = densedunet121
# default_configs['densedunet121'] = dunet_config
# splits['densedunet121'] = _densedunet_split

# def densedunet161(data, config): return _densedunet(densenet161, data, config)
# model_funcs['densedunet161'] = densedunet161
# default_configs['densedunet161'] = dunet_config
# splits['densedunet161'] = _densedunet_split

# def densedunet169(data, config): return _densedunet(densenet169, data, config)
# model_funcs['densedunet169'] = densedunet169
# default_configs['densedunet169'] = dunet_config
# splits['densedunet169'] = _densedunet_split

# def densedunet201(data, config): return _densedunet(densenet201, data, config)
# model_funcs['densedunet201'] = densedunet201
# default_configs['densedunet201'] = dunet_config
# splits['densedunet201'] = _densedunet_split


### `SE-ResDUneXt [50,101]`

In [18]:
from fastai.vision.models.cadene_models import se_resnext50_32x4d, se_resnext101_32x4d
from fastai.vision.models.cadene_models import model_meta

In [19]:
#export
_se_meta = model_meta[se_resnext50_32x4d]
_se_cut, _se_split = _se_meta['cut'], _se_meta['split']

In [20]:
#export
def _seresdunext(arch, data, config):
    "Returns a resdunet model for a arch from data and final config"
    pretrained, cut = config.pop("pretrained"), config.pop("cut")
    try:    size = data.train_ds[0][0].size
    except: size = next(iter(data.train_dl))[0].shape[-2:]
    body = create_body(arch, pretrained, cut=_se_cut)
    model = DynamicUnet(body, n_classes=data.c, img_size=size, **config)
    return model

In [21]:
#export
def seresdunext50(data, config): return _seresdunext(se_resnext50_32x4d, data, config)
model_funcs['seresdunext50'] = seresdunext50
default_configs['seresdunext50'] = dunet_config
splits['seresdunext50'] = _se_split

def seresdunext101(data, config): return _seresdunext(se_resnext101_32x4d, data, config)
model_funcs['seresdunext101'] = seresdunext101
default_configs['seresdunext101'] = dunet_config
splits['seresdunext101'] = _se_split

In [22]:
model, split,_ = get_model(name="seresdunext50", data=data, config={"self_attention":True})

### `DeepLab v3+ [ResNet50, ResNet101]`

In [27]:
from torchvision.models.segmentation import deeplabv3_resnet50, deeplabv3_resnet101

In [30]:
model = deeplabv3_resnet101(True)

In [84]:
model = torch.hub.load('pytorch/vision', 'deeplabv3_resnet101', pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/master.zip" to /home/turgutluk/.cache/torch/hub/master.zip


ImportError: cannot import name 'wide_resnet50_2' from 'torchvision.models.resnet' (/home/turgutluk/.conda/envs/my_fastai/lib/python3.7/site-packages/torchvision/models/resnet.py)

In [85]:
import torchvision

In [86]:
torchvision.__version__

'0.3.0'

### `tests`

In [66]:
from local.test import *

In [67]:
model_funcs

{'resdunet18': <function __main__.resdunet18(data, config)>,
 'resdunet34': <function __main__.resdunet34(data, config)>,
 'resdunet50': <function __main__.resdunet50(data, config)>,
 'resdunet101': <function __main__.resdunet101(data, config)>,
 'resdunet152': <function __main__.resdunet152(data, config)>,
 'densedunet121': <function __main__.densedunet121(data, config)>,
 'densedunet161': <function __main__.densedunet161(data, config)>,
 'densedunet169': <function __main__.densedunet169(data, config)>,
 'densedunet201': <function __main__.densedunet201(data, config)>}

In [68]:
splits

{'resdunet18': <function fastai.vision.learner._resnet_split(m: torch.nn.modules.module.Module)>,
 'resdunet34': <function fastai.vision.learner._resnet_split(m: torch.nn.modules.module.Module)>,
 'resdunet50': <function fastai.vision.learner._resnet_split(m: torch.nn.modules.module.Module)>,
 'resdunet101': <function fastai.vision.learner._resnet_split(m: torch.nn.modules.module.Module)>,
 'resdunet152': <function fastai.vision.learner._resnet_split(m: torch.nn.modules.module.Module)>,
 'densedunet121': <function __main__._densedunet_split(m: torch.nn.modules.module.Module)>,
 'densedunet161': <function __main__._densedunet_split(m: torch.nn.modules.module.Module)>,
 'densedunet169': <function __main__._densedunet_split(m: torch.nn.modules.module.Module)>,
 'densedunet201': <function __main__._densedunet_split(m: torch.nn.modules.module.Module)>}

### `resdunet`

In [29]:
model, split, config = get_model(name="resdunet18", data=data, config={"self_attention":False})
learn = Learner(data, model)
learn.metrics = [partial(foreground_acc, void_code=31)]
learn = learn.split(split)
if config.get("pretrained"): learn.freeze()
learn.fit_one_cycle(3)
learn.unfreeze()
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,foreground_acc,time
0,1.777493,2.456983,0.418664,00:08
1,1.364526,1.005066,0.716271,00:08
2,0.943640,0.756950,0.806386,00:08


epoch,train_loss,valid_loss,foreground_acc,time
0,0.832981,0.781028,0.807179,00:09
1,0.761613,0.639291,0.835867,00:08
2,0.666020,0.550050,0.849063,00:08


In [30]:
res = learn.recorder.metrics[-1][0].item(); print(res)
assert res > 0.75

0.8490628004074097


### `densedunet`

In [81]:
# model, split, config = get_model(name="densedunet161", data=data, config={"self_attention":False})
# learn = Learner(data, model)
# learn.metrics = [partial(foreground_acc, void_code=31)]
# learn = learn.split(split)
# if config.get("pretrained"): learn.freeze()
# learn.fit_one_cycle(3)
# learn.unfreeze()
# learn.fit_one_cycle(3)

In [82]:
# res = learn.recorder.metrics[-1][0].item(); print(res)
# assert res > 0.75

### `seresdunext`

In [25]:
model, split, config = get_model(name="seresdunext50", data=data, config={"self_attention":True})
learn = Learner(data, model)
learn.metrics = [partial(foreground_acc, void_code=31)]
learn = learn.split(split)
if config.get("pretrained"): learn.freeze()
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,foreground_acc,time
0,9.945091,2.052992,0.463276,00:41
1,3.553678,1.263506,0.591455,00:41
2,1.070011,0.958146,0.662554,00:42


In [24]:
res = learn.recorder.metrics[-1][0].item(); print(res)
assert res > 0.75

0.8222544193267822


### export

In [26]:
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_script.ipynb.
Converted 02_scheduler.ipynb.
Converted 03_callbacks.ipynb.
Converted 10_segmentation_dataset.ipynb.
Converted 11_segmentation_losses_mulitlabel.ipynb.
Converted 11b_segmentation_losses_binary.ipynb.
Converted 12_segmentation_metrics.ipynb.
Converted 13_segmentation_models.ipynb.
Converted segmentation_training.ipynb.


### fin